<a href="https://colab.research.google.com/github/Python-Financial-Analyst/pyfian_dev/blob/main/notebooks/fixed_income/04_bootstrapping.ipynb" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/>
</a>

# Comparing Bond Returns

## Learning Objectives

- Compare different metrics for bond returns. 

## Yield to Maturity (YTM)

Yield to Maturity (YTM) is the internal rate of return (IRR) of a bond assuming it is held until maturity and all coupon payments are reinvested at the same rate. It represents the annualized return an investor can expect to earn if the bond is held to maturity.

YTM can be found by solving the following equation for \(YTM\):

$$
P = \sum_{t=1}^{N} \frac{C}{(1 + YTM)^t} + \frac{F}{(1 + YTM)^N}
$$

where:  
- $P$ = current price of the bond  
- $C$ = coupon payment  
- $F$ = face value  
- $N$ = number of periods until maturity  

## Current Yield

Current Yield is a simple measure of the annual income (coupon) of a bond relative to its current price. It does not account for capital gains/losses due to discount or premium pricing.

$$
\text{Current Yield} = \frac{\text{Annual Coupon Payment}}{\text{Current Bond Price}} \times 100
$$

## IRR (Internal Rate of Return)

IRR is the discount rate that makes the net present value (NPV) of all cash flows from a bond equal to zero. For a bond:

$$
NPV = \sum_{t=1}^{N} \frac{CF_t}{(1 + IRR)^t} - P_0 = 0
$$

where $CF_t$ are the cash flows, and $P_0$ is the initial investment (price). IRR reflects the true annualized return considering the timing and size of cash flows.

## XIRR (Extended IRR)

XIRR is similar to IRR but accounts for irregular time periods between cash flows. This is especially useful if coupon payments or settlement dates are not perfectly regular.  

In practice, XIRR is calculated numerically using all actual dates of cash flows.

## AER (Annual Equivalent Rate)

The Annual Equivalent Rate (AER) represents the annualized interest rate that accounts for compounding over the year. It allows investors to compare returns on different investment products on a consistent annual basis, regardless of how frequently interest is compounded (monthly, quarterly, etc.).

The formula for AER is:

$$
\text{AER} = \left(1 + \frac{i}{n}\right)^n - 1
$$

where:  
- $i$ = nominal interest rate  
- $n$ = number of compounding periods per year  

Using AER, investors can directly compare accounts or bonds that compound at different frequencies, ensuring a like-for-like comparison.

## BEY (Bond Equivalent Yield)

The Bond Equivalent Yield (BEY) converts the yield on a bond, often quoted on a discount or semi-annual basis, into an annualized yield that is comparable to standard coupon-bearing bonds. This is particularly useful for short-term securities like Treasury bills or zero-coupon bonds.

For a discount bond (e.g., a T-bill), BEY is calculated as:

$$
\text{BEY} = \frac{F - P}{P} \cdot \frac{365}{t}
$$

where:  
- $F$ = face value of the bond  
- $P$ = purchase price  
- $t$ = days to maturity  

For a semi-annual coupon bond, BEY can also be expressed as:

$$
\text{BEY} = 2 \left[ \left(1 + \frac{YTM_{\text{semi-annual}}}{2}\right)^2 - 1 \right]
$$

Using BEY allows investors to compare yields of bonds with different maturities or coupon structures on a consistent annual basis, making it easier to assess relative value across instruments.

## PVBP (Price Value of a Basis Point)

PVBP measures the price change of a bond for a 1 basis point (0.01%) change in yield. It is a risk metric used to assess interest rate sensitivity.

$$
PVBP = \frac{\Delta P}{\Delta y} \cdot 0.0001
$$

where $\Delta P$ is the change in bond price corresponding to a 1 bp change in yield.

In [14]:
import pandas as pd
from pyfian.fixed_income.fixed_rate_bond import FixedRateBullet
import IPython.display as ipd

# Define two example bonds
bond1 = FixedRateBullet(
    issue_dt="2025-01-01",
    maturity="2030-01-01",
    cpn=10,
    cpn_freq=2,  # Semi-annual coupon
    notional=100,
    settlement_date="2025-12-01",
    price=100,
)

bond2 = FixedRateBullet(
    issue_dt="2025-01-01",
    maturity="2030-01-01",
    cpn=10,
    cpn_freq=4,  # Quarterly coupon
    notional=100,
    settlement_date="2025-12-01",
    price=100,
)

bonds = [bond1, bond2]
bond_names = ["Bond 1", "Bond 2"]

# List of attributes
attributes = [
    "Instrument",
    "Issue Date",
    "Maturity Date",
    "Settlement Date",
    "Notional",
    "Coupon (%)",
    "Coupon Frequency",
    "Day Count Convention",
    "Price (Input)",
    "Yield to Maturity (%)",
    "Current Yield (%)",
    "PVBP",
    "IRR",
    "XIRR",
    "BEY (%)",
    "AER (%)",
    "Macaulay Duration (Years)",
    "Modified Duration (Years)",
    "Convexity",
]


# Helper functions for BEY and AER
def calculate_bey(ytm, freq):
    """Bond Equivalent Yield for a semi-annual or other periodic bond"""
    return 0


def calculate_aer(ytm, freq):
    """Annual Equivalent Rate"""
    return 0


# Build table dictionary
table_dict = {"Attribute": attributes}

for b_name, b in zip(bond_names, bonds):
    clean_price = b.get_price()
    accrued_interest = b.accrued_interest()
    dirty_price = clean_price + accrued_interest
    ytm = b.get_yield_to_maturity()
    current_yield = (b.cpn / clean_price) * 100
    macaulay_duration = b.macaulay_duration()
    modified_duration = b.modified_duration()
    convexity = b.convexity()
    pvbp = modified_duration * clean_price / 10000

    bey = calculate_bey(ytm, b.cpn_freq)
    aer = calculate_aer(ytm, b.cpn_freq)

    values = [
        b.__class__.__name__,  # Instrument
        b.issue_dt,  # Issue Date
        b.maturity,  # Maturity Date
        b.get_settlement_date(),  # Settlement Date
        b.notional,  # Notional
        b.cpn,  # Coupon (%)
        b.cpn_freq,  # Coupon Frequency
        b.day_count_convention.__class__.__name__,  # Day Count
        f"{clean_price:.2f}",  # Price input
        f"{ytm * 100:.2f} ★",  # YTM
        f"{current_yield:.2f} ★",  # Current Yield
        f"{pvbp:.4f} ★",  # PVBP
        "0 ★",  # IRR placeholder
        "0 ★",  # XIRR placeholder
        f"{bey:.2f} ★",  # BEY
        f"{aer:.2f} ★",  # AER
        f"{macaulay_duration:.2f} ★",  # Macaulay Duration
        f"{modified_duration:.2f} ★",  # Modified Duration
        f"{convexity:.4f} ★",  # Convexity
    ]

    table_dict[b_name] = values

# Convert to DataFrame
df = pd.DataFrame(table_dict)

# Display table without index
ipd.display(df.style.hide(axis="index"))

Attribute,Bond 1,Bond 2
Instrument,FixedRateBullet,FixedRateBullet
Issue Date,2025-01-01 00:00:00,2025-01-01 00:00:00
Maturity Date,2030-01-01 00:00:00,2030-01-01 00:00:00
Settlement Date,2025-12-01 00:00:00,2025-12-01 00:00:00
Notional,100,100
Coupon (%),10,10
Coupon Frequency,2,4
Day Count Convention,DayCountActualActualBond,DayCountActualActualBond
Price (Input),100.00,100.00
Yield to Maturity (%),11.29 ★,11.21 ★
